# Scope del Proyecto

## Objetivo general:

* Analizar la satisfacción de los pasajeros de las aerolíneas y determinar los factores que tienen el mayor impacto en su satisfacción.

## Objetivos específicos:

* Identificar las variables que están más correlacionadas con la satisfacción de los pasajeros.
* Analizar la relación entre la satisfacción de los pasajeros y los diferentes aspectos del servicio de la aerolínea, como la comodidad del asiento, el entretenimiento a bordo, la limpieza, etc.
* Determinar si hay diferencias significativas en la satisfacción de los pasajeros según la aerolínea, el tipo de viaje (nacional/internacional), la clase de viaje, el destino, etc.
* Identificar patrones en la satisfacción de los pasajeros a lo largo del tiempo.

## Preguntas Planteadas
1. ¿Cuantos clientes leales tuvieron vuelos satisfactorios?
2. ¿A qué sexo (masculino y femenino) le costó más agendar un vuelo en línea?
3. ¿Cuáles son los factores que tienen el mayor impacto en la satisfacción de los pasajeros?
4. ¿Hay alguna diferencia en la satisfacción de los pasajeros entre las diferentes aerolíneas?
5. ¿La satisfacción de los pasajeros difiere según el tipo de viaje o la clase de viaje?
6. ¿Existe algún patrón en la satisfacción de los pasajeros a lo largo del tiempo?

# Exploración de datos

In [1]:
# Importación de librerias utilizadas en el proyecto
import numpy as np
import pandas as pd
import boto3
import psycopg2
import configparser
import io
from io import BytesIO
import random

In [23]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [3]:
# Se prepara para la lectura del dataset almacenado en S3
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [4]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

airlineairports
airlinesatisfactiondataset


In [5]:
S3_BUCKET_AIRLINES = 'airlinesatisfactiondataset'
FILE_NAME = 'data/airline_satisfaction.csv'

In [6]:
S3_BUCKET_AIRLPORTS = 'airlineairports'
FILE_NAME = 'data/airports.csv'

In [7]:
#extraemos todo lo que está en el bucket para los datos de satisfaccion
remoteFileListAirline = []
for objt in s3.Bucket(S3_BUCKET_AIRLINES).objects.all():
    remoteFileListAirline.append(objt.key)

remoteFileListAirline

['data/', 'data/airline_satisfaction.csv']

In [8]:
#extraemos todo lo que está en el bucket para los datos de aeropuertos
remoteFileListAirports = []
for objt in s3.Bucket(S3_BUCKET_AIRLPORTS).objects.all():
    remoteFileListAirports.append(objt.key)

remoteFileListAirports

['data/', 'data/airports.csv']

## Se lee el archivo con datos de satisfacción de los aeropuertos

In [9]:
for remoteFile in remoteFileListAirports:
    try:
        file = s3.Bucket(S3_BUCKET_AIRLPORTS).Object(remoteFile).get()
        data = file['Body'].read()
        df_airports = pd.read_csv(BytesIO(data))
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df_airports.head()

No es un archivo.
No columns to parse from file


,idAirport,airportName,airportCode
0,1,Ukhta Airport,UCT
1,2,Aleknagik / New Airport,WKK
2,3,Guangzhou Baiyun International Airport,CAN
3,4,Leipzig/Halle Airport,LEJ
4,5,Charleston Air Force Base-International Airport,CHS


## Se lee el archivo con datos de satisfacción de las aerolineas

In [10]:
for remoteFile in remoteFileListAirline:
    try:
        file = s3.Bucket(S3_BUCKET_AIRLINES).Object(remoteFile).get()
        data = file['Body'].read()
        df = pd.read_csv(BytesIO(data))
        # Se elimina la primera columna del dataframe (era el número de fila)
        del df[df.columns[0]]
    except Exception as ex:
        print("No es un archivo.")
        print(ex)

df.head()

No es un archivo.
No columns to parse from file


,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,106966,Female,Loyal Customer,40,Business travel,Eco,1036,2,3,3,...,2,1,1,2,1,2,2,26,32.0,neutral or dissatisfied
1,54152,Female,Loyal Customer,34,Business travel,Business,1569,2,2,2,...,3,3,4,3,3,3,3,0,0.0,satisfied
2,26463,Female,Loyal Customer,54,Personal Travel,Eco,925,3,4,4,...,4,4,4,4,3,4,2,0,1.0,neutral or dissatisfied
3,46257,Female,Loyal Customer,69,Personal Travel,Eco,624,3,1,3,...,4,4,3,4,1,4,3,36,33.0,neutral or dissatisfied
4,33133,Male,Loyal Customer,48,Personal Travel,Eco,102,4,4,4,...,1,4,3,5,5,5,1,1,4.0,satisfied


## Se unen las distintas fuentes de información

In [15]:
# Obtiene una lista de los idAirport del dataset de aeropuertos
airport_ids = list(df_airports['idAirport'])

# Agrega una nueva columna "idAirport" al dataset de satisfacción y llena cada fila con un idAirport aleatorio
df['idAirport'] = [random.choice(airport_ids) for _ in range(len(df))]

In [17]:
# Dataset de satisfacción con la nueva columna
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,idAirport
0,106966,Female,Loyal Customer,40,Business travel,Eco,1036,2,3,3,...,1,1,2,1,2,2,26,32.0,neutral or dissatisfied,866
1,54152,Female,Loyal Customer,34,Business travel,Business,1569,2,2,2,...,3,4,3,3,3,3,0,0.0,satisfied,622
2,26463,Female,Loyal Customer,54,Personal Travel,Eco,925,3,4,4,...,4,4,4,3,4,2,0,1.0,neutral or dissatisfied,908
3,46257,Female,Loyal Customer,69,Personal Travel,Eco,624,3,1,3,...,4,3,4,1,4,3,36,33.0,neutral or dissatisfied,239
4,33133,Male,Loyal Customer,48,Personal Travel,Eco,102,4,4,4,...,4,3,5,5,5,1,1,4.0,satisfied,588


## Análisis exploratorio del dataset de satisfacción

In [11]:
# Se obtienen estadísticas descriptivas de variables numéricas
print(df.describe())

                  id           Age  Flight Distance  Inflight wifi service  \
count   39999.000000  39999.000000     39999.000000           39999.000000   
mean    64883.149154     39.393060      1191.004425               2.728768   
std     37471.739749     15.094959       994.866002               1.326630   
min         2.000000      7.000000        31.000000               0.000000   
25%     32485.500000     27.000000       416.000000               2.000000   
50%     64951.000000     40.000000       846.000000               3.000000   
75%     97285.500000     51.000000      1739.000000               4.000000   
max    129880.000000     85.000000      4983.000000               5.000000   

       Departure/Arrival time convenient  Ease of Online booking  \
count                       39999.000000            39999.000000   
mean                            3.054601                2.754969   
std                             1.521310                1.396526   
min                      

In [12]:
# Se obtiene la distribución de los valores en la variable "satisfaction"
print(df["satisfaction"].value_counts())

neutral or dissatisfied    22732
satisfied                  17267
Name: satisfaction, dtype: int64


In [13]:
# Se calcula la correlación entre variables numéricas
print(df.corr())

                                         id       Age  Flight Distance  \
id                                 1.000000  0.018632         0.092981   
Age                                0.018632  1.000000         0.103459   
Flight Distance                    0.092981  0.103459         1.000000   
Inflight wifi service             -0.022387  0.015945         0.003062   
Departure/Arrival time convenient -0.000570  0.041585        -0.022547   
Ease of Online booking             0.014507  0.021882         0.060478   
Gate location                      0.003063 -0.001064         0.002079   
Food and drink                    -0.003640  0.022470         0.052078   
Online boarding                    0.056004  0.209083         0.215556   
Seat comfort                       0.044269  0.156752         0.158683   
Inflight entertainment            -0.006200  0.074431         0.123347   
On-board service                   0.051124  0.054516         0.111838   
Leg room service                   0.0

# Modelo Dimensional Propuesto

### Tabla de Hechos (Satisfacción del Cliente)

* idCliente (clave foránea)
* idViaje (clave foránea)
* idServicio (clave foránea)
* idAeropuerto (clave foránea)
* satisfaction
* DepartureDelay
* ArrivalDelay

### Dimensión Cliente

* IdCliente (clave primaria)
* Gender
* CustomerType
* age

### Dimensión Viaje

* idViaje (clave primaria)
* TypeTravel
* Class
* FlightDistance

### Dimensión Servicio

* idServicio (clave primaria)
* InflightWifiService
* DepartureArrivalTimeConvenient
* EaseOnlineBooking
* GateLocation
* FoodAndDrink
* OnlineBoarding
* SeatComfort
* InflightEntertainment
* OnboardService
* LegRoomService
* BaggageHandling
* CheckinService
* InflightService
* Cleanliness

### Dimensión Aeropuerto

* idAeropuerto
* airportName
* airportCode

## Preparando la base de datos en RDS para la carga de las dimensiones
## y tabla de hechos

In [18]:
# Se crea una instancia de RDS
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')

In [21]:
rdsIdentifier = 'al-satisfaction-db' #nombre de la instancia

In [24]:
# Se crea una instancia de base de datos en RDS (Se comenta porque solamente se utiliza una vez)
""" try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.") """

{'DBInstance': {'DBInstanceIdentifier': 'al-satisfaction-db', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'postgres', 'DBName': 'airline_satisfaction', 'AllocatedStorage': 10, 'PreferredBackupWindow': '06:32-07:02', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0f01d4870c2f2ad6f', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres14', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-05e05c3803e3efa5c', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-00333d3085e894e2e', 'SubnetAvailabilityZone': {'Name': 'us-east-1a'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-087b01558053e95f4', 'SubnetAvailabilityZone': {'Name': 'us-east-1f'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIde

In [25]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['al-satisfaction-db']
